In [68]:
import os
import pandas as pd
import numpy as np
import sklearn
import csv

In [69]:
df = pd.read_csv('Charades_vu17_train.csv')

In [70]:
df.head()

,id,subject,scene,quality,relevance,verified,script,objects,descriptions,actions,length
0,46GP8,HR43,Kitchen,6.0,7.0,Yes,A person cooking on a stove while watching som...,food;stove;window,A person cooks food on a stove before looking ...,c092 11.90 21.20;c147 0.00 12.60,24.83
1,N11GT,0KZ7,Stairs,6.0,7.0,Yes,"One person opens up a folded blanket, then sne...",blanket;broom;floor,Person at the bottom of the staircase shakes a...,c098 8.60 14.20;c075 0.00 11.70;c127 0.00 15.2...,18.33
2,0IH69,6RE8,Bedroom,6.0,5.0,Yes,A person is seen leaving a cabinet. They then ...,book;box;cabinet;shelf,A person is standing in a bedroom. They walk o...,NaN,30.25
3,KRF68,YA10,Laundry room,6.0,7.0,Yes,A person runs into their laundry room. They gr...,clothes;door;phone,A person runs in and shuts door. The person gr...,c018 22.60 27.80;c141 4.10 9.60;c148 10.30 25....,30.33
4,MJO7C,6RE8,Kitchen,6.0,6.0,Yes,A person runs into their pantry holding a bott...,cup;phone,A person runs in place while holding a bottle ...,c015 0.00 32.00;c107 0.00 32.00,31.38


In [71]:
df = df.dropna()
# df = df[df.quality >=6 ]
# df = df[df.relevance == 7] #  和 script 的相關性，但我們不需要管 script ，所以用不到
df.head()
df['origin_index'] = df.index.values

In [72]:
df = df.reset_index(drop=True)
df.head()

,id,subject,scene,quality,relevance,verified,script,objects,descriptions,actions,length,origin_index
0,46GP8,HR43,Kitchen,6.0,7.0,Yes,A person cooking on a stove while watching som...,food;stove;window,A person cooks food on a stove before looking ...,c092 11.90 21.20;c147 0.00 12.60,24.83,0
1,N11GT,0KZ7,Stairs,6.0,7.0,Yes,"One person opens up a folded blanket, then sne...",blanket;broom;floor,Person at the bottom of the staircase shakes a...,c098 8.60 14.20;c075 0.00 11.70;c127 0.00 15.2...,18.33,1
2,KRF68,YA10,Laundry room,6.0,7.0,Yes,A person runs into their laundry room. They gr...,clothes;door;phone,A person runs in and shuts door. The person gr...,c018 22.60 27.80;c141 4.10 9.60;c148 10.30 25....,30.33,3
3,MJO7C,6RE8,Kitchen,6.0,6.0,Yes,A person runs into their pantry holding a bott...,cup;phone,A person runs in place while holding a bottle ...,c015 0.00 32.00;c107 0.00 32.00,31.38,4
4,S6MPZ,EA2K,Home Office / Study (A room in a house used fo...,6.0,6.0,Yes,A person is eating at the desk and lying the p...,chair;desk;food;phone;plate;snacks;table,A person in a blue shirt is eating snacks off ...,c009 0.00 4.30;c011 0.00 39.00;c015 0.00 39.00...,38.46,5


In [73]:
dat_id = df['id']
dat_actions = df['actions']
dat_origin_index = df['origin_index']
dat = pd.concat([dat_id, dat_origin_index, dat_actions], axis=1)
split_arr = dat['actions'].str.split(';')

In [74]:
large_arr = []
for element in enumerate(split_arr):
    arr = []
    for i in element[1]:
        k = i.split(" ")
        d = k[0]
        arr.append(d)
    large_arr.append(arr)
dat['split'] = large_arr

In [75]:
dat = dat.drop('actions', 1)

In [76]:
dat.head()

,id,origin_index,split
0,46GP8,0,"[c092, c147]"
1,N11GT,1,"[c098, c075, c127, c153]"
2,KRF68,3,"[c018, c141, c148, c006, c002, c150, c000]"
3,MJO7C,4,"[c015, c107]"
4,S6MPZ,5,"[c009, c011, c015, c019, c156, c059, c061, c06..."


In [77]:
str_actions = dat['split']
int_actions = []
for action in str_actions:
    arr = []
    for act in action:
        a = int(act[1:])
        arr.append(a)
    int_actions.append(arr)

In [78]:
with open('train_actions_raw.csv', 'w') as ofs:
    for act in str_actions:
        mystr = ",".join(act)
        ofs.write(mystr + '\n')